## model/region_proposal_network.py 

In [ ]:
class RegionProposalNetwork(nn.Module):
    """Region Proposal Network introduced in Faster R-CNN.
    This is Region Proposal Network introduced in Faster R-CNN [#]_.
    This takes features extracted from images and propose
    class agnostic bounding boxes around "objects".
    .. [#] Shaoqing Ren, Kaiming He, Ross Girshick, Jian Sun. \
    Faster R-CNN: Towards Real-Time Object Detection with \
    Region Proposal Networks. NIPS 2015.
    Args:
        in_channels (int): The channel size of input.
        mid_channels (int): The channel size of the intermediate tensor.
        ratios (list of floats): This is ratios of width to height of
            the anchors.
        anchor_scales (list of numbers): This is areas of anchors.
            Those areas will be the product of the square of an element in
            :obj:`anchor_scales` and the original area of the reference
            window.
        feat_stride (int): Stride size after extracting features from an
            image.
        initialW (callable): Initial weight value. If :obj:`None` then this
            function uses Gaussian distribution scaled by 0.1 to
            initialize weight.
            May also be a callable that takes an array and edits its values.
        proposal_creator_params (dict): Key valued paramters for
            :class:`model.utils.creator_tools.ProposalCreator`.
    .. seealso::
        :class:`~model.utils.creator_tools.ProposalCreator`
    """
    1.计算回归坐标偏移量
    2.计算anchors前景概率
    3.调用ProposalCreator和使用NMS得出2000个近似目标框的坐标

    def __init__(
            self, in_channels=512, mid_channels=512, ratios=[0.5, 1, 2],
            anchor_scales=[8, 16, 32], feat_stride=16,
            proposal_creator_params=dict(),
    ):
        super(RegionProposalNetwork, self).__init__()
        #调用generate_anchor_base（）函数，生成左上角9个anchor_base
        self.anchor_base = generate_anchor_base(
            anchor_scales=anchor_scales, ratios=ratios)
        self.feat_stride = feat_stride
        self.proposal_layer = ProposalCreator(self, **proposal_creator_params) #输出2000roi
        n_anchor = self.anchor_base.shape[0] #9个
        self.conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
        self.score = nn.Conv2d(mid_channels, n_anchor * 2, 1, 1, 0)
        self.loc = nn.Conv2d(mid_channels, n_anchor * 4, 1, 1, 0)
        #作归一化处理
        normal_init(self.conv1, 0, 0.01)
        normal_init(self.score, 0, 0.01)
        normal_init(self.loc, 0, 0.01)

    def forward(self, x, img_size, scale=1.):
        """Forward Region Proposal Network.
        Here are notations.
        * :math:`N` is batch size.
        * :math:`C` channel size of the input.
        * :math:`H` and :math:`W` are height and witdh of the input feature.
        * :math:`A` is number of anchors assigned to each pixel.
        Args:
            x (~torch.autograd.Variable): The Features extracted from images.
                Its shape is :math:`(N, C, H, W)`.
            img_size (tuple of ints): A tuple :obj:`height, width`,
                which contains image size after scaling.
            scale (float): The amount of scaling done to the input images after
                reading them from files.
        Returns:
            (~torch.autograd.Variable, ~torch.autograd.Variable, array, array, array):
            This is a tuple of five following values.
            * **rpn_locs**: Predicted bounding box offsets and scales for \
                anchors. Its shape is :math:`(N, H W A, 4)`.
            * **rpn_scores**:  Predicted foreground scores for \
                anchors. Its shape is :math:`(N, H W A, 2)`.
            * **rois**: A bounding box array containing coordinates of \
                proposal boxes.  This is a concatenation of bounding box \
                arrays from multiple images in the batch. \
                Its shape is :math:`(R', 4)`. Given :math:`R_i` predicted \
                bounding boxes from the :math:`i` th image, \
                :math:`R' = \\sum _{i=1} ^ N R_i`.
            * **roi_indices**: An array containing indices of images to \
                which RoIs correspond to. Its shape is :math:`(R',)`.
            * **anchor**: Coordinates of enumerated shifted anchors. \
                Its shape is :math:`(H W A, 4)`.
        """
        n, _, hh, ww = x.shape #(batch_size，512,H/16,W/16），其中H，W分别为原图的高和宽
        anchor = _enumerate_shifted_anchor(
            np.array(self.anchor_base),
            self.feat_stride, hh, ww) #在9个base_anchor基础上生成hh*ww*9个anchor，对应到原图坐标

        n_anchor = anchor.shape[0] // (hh * ww)  #hh*ww*9/hh*ww=9
        h = F.relu(self.conv1(x)) #512个3x3卷积(512, H/16,W/16)

        rpn_locs = self.loc(h) #n_anchor（9）*4个1x1卷积，回归坐标偏移量。（9*4，hh,ww）

        #转换为（n，hh，ww，9*4）后变为（n，hh*ww*9，4）
        rpn_locs = rpn_locs.permute(0, 2, 3, 1).contiguous().view(n, -1, 4)
        rpn_scores = self.score(h) #n_anchor（9）*2个1x1卷积，回归类别。（9*2，hh,ww）
        rpn_scores = rpn_scores.permute(0, 2, 3, 1).contiguous() #转换为（n，hh，ww，9*2）
        rpn_softmax_scores = F.softmax(rpn_scores.view(n, hh, ww, n_anchor, 2), dim=4) #计算softmax
        rpn_fg_scores = rpn_softmax_scores[:, :, :, :, 1].contiguous() #得到前景的分类概率
        rpn_fg_scores = rpn_fg_scores.view(n, -1) #得到所有anchor的前景分类概率
        rpn_scores = rpn_scores.view(n, -1, 2) #得到每一张feature map上所有anchor的网络输出值

        rois = list()
        roi_indices = list()
        for i in range(n): #n为batch_size数
            
        # 调用ProposalCreator函数， rpn_locs维度（hh*ww*9，4），rpn_fg_scores维度为（hh*ww*9），
        #anchor的维度为（hh*ww*9，4）， img_size的维度为（3，H，W），H和W是经过数据预处理后的。
        #计算（H/16）x(W/16)x9(大概20000)个anchor属于前景的概率，取前12000个并经过NMS得到2000个近似目标框的坐标。
  
            roi = self.proposal_layer(
                rpn_locs[i].cpu().data.numpy(), #位置信息
                rpn_fg_scores[i].cpu().data.numpy(), #前景概率
                anchor, img_size,
                scale=scale)
            batch_index = i * np.ones((len(roi),), dtype=np.int32)
            rois.append(roi)
            roi_indices.append(batch_index)

        rois = np.concatenate(rois, axis=0) #rois为所有batch_size的roi
        roi_indices = np.concatenate(roi_indices, axis=0) #按行拼接
        #rpn_locs的维度（hh*ww*9，4），rpn_scores维度为（hh*ww*9，2）， 
        #rois的维度为（2000,4），roi_indices用不到，anchor的维度为（hh*ww*9，4）
        return rpn_locs, rpn_scores, rois, roi_indices, anchor 


def _enumerate_shifted_anchor(anchor_base, feat_stride, height, width):
    #在整个特征图生成所有的anchor，对应回原图大小

    import numpy as xp
    shift_y = xp.arange(0, height * feat_stride, feat_stride)#纵向偏移量（0，16，32，...）
    shift_x = xp.arange(0, width * feat_stride, feat_stride)# 横向偏移量（0，16，32，...）
     #shift_x = [[0，16，32，..],[0，16，32，..],[0，16，32，..]...],
     #shift_y = [[0，0，0，..],[16，16，16，..],[32，32，32，..]...],
    #就是形成了一个纵横向偏移量的矩阵，也就是特征图的每一点都能够通过这个矩阵找到映射在原图中的具体位置！
    shift_x, shift_y = xp.meshgrid(shift_x, shift_y)
    #产生偏移坐标对，一个朝x方向，一个朝y方向偏移。
    shift = xp.stack((shift_y.ravel(), shift_x.ravel(),
                      shift_y.ravel(), shift_x.ravel()), axis=1)

    A = anchor_base.shape[0] #A=9
    K = shift.shape[0] #读取特征图中元素的总个数
    anchor = anchor_base.reshape((1, A, 4)) + \
             shift.reshape((1, K, 4)).transpose((1, 0, 2))
    anchor = anchor.reshape((K * A, 4)).astype(np.float32)
    return anchor


def _enumerate_shifted_anchor_torch(anchor_base, feat_stride, height, width):
    # Enumerate all shifted anchors:
    #
    # add A anchors (1, A, 4) to
    # cell K shifts (K, 1, 4) to get
    # shift anchors (K, A, 4)
    # reshape to (K*A, 4) shifted anchors
    # return (K*A, 4)

    # !TODO: add support for torch.CudaTensor
    # xp = cuda.get_array_module(anchor_base)
    import torch as t
    shift_y = t.arange(0, height * feat_stride, feat_stride)
    shift_x = t.arange(0, width * feat_stride, feat_stride)
    shift_x, shift_y = xp.meshgrid(shift_x, shift_y)
    shift = xp.stack((shift_y.ravel(), shift_x.ravel(),
                      shift_y.ravel(), shift_x.ravel()), axis=1)

    A = anchor_base.shape[0]
    K = shift.shape[0]
    anchor = anchor_base.reshape((1, A, 4)) + \
             shift.reshape((1, K, 4)).transpose((1, 0, 2))
    anchor = anchor.reshape((K * A, 4)).astype(np.float32)
    return anchor


def normal_init(m, mean, stddev, truncated=False):
    """
    weight initalizer: truncated normal and random normal.
    """
    # x is a parameter
    if truncated:
        m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)  # not a perfect approximation
    else:
        m.weight.data.normal_(mean, stddev)
        m.bias.data.zero_()

## Faster RCNN

In [ ]:
def decom_vgg16():
    # the 30th layer of features is relu of conv5_3
    if opt.caffe_pretrain: #是否使用caffe预训练
        model = vgg16(pretrained=False)
        if not opt.load_path:
            model.load_state_dict(t.load(opt.caffe_pretrain_path))
    else:
        model = vgg16(not opt.load_path)

    features = list(model.features)[:30] #加载预训练模型vgg16的conv5_3之前的部分
    classifier = model.classifier

    classifier = list(classifier)
    del classifier[6] #删除最后分1000类
    if not opt.use_drop: #删除两个dropout
        del classifier[5]
        del classifier[2]
    classifier = nn.Sequential(*classifier)

    for layer in features[:10]: #冻结vgg16前2个stage,不进行反向传播
        for p in layer.parameters():
            p.requires_grad = False

    return nn.Sequential(*features), classifier #拆分为特征提取网络和分类网络


class FasterRCNNVGG16(FasterRCNN):
    """Faster R-CNN based on VGG-16.
    For descriptions on the interface of this model, please refer to
    :class:`model.faster_rcnn.FasterRCNN`.
    Args:
        n_fg_class (int): The number of classes excluding the background.
        ratios (list of floats): This is ratios of width to height of
            the anchors.
        anchor_scales (list of numbers): This is areas of anchors.
            Those areas will be the product of the square of an element in
            :obj:`anchor_scales` and the original area of the reference
            window.
    """
    #分别对特征VGG16的特征提取部分、分类部分、RPN网络、VGG16RoIHead网络进行了实例化
    feat_stride = 16  # downsample 16x for output of conv5 in vgg16

    def __init__(self,
                 n_fg_class=20,
                 ratios=[0.5, 1, 2],
                 anchor_scales=[8, 16, 32]
                 ): #总类别数为20类，三种尺度三种比例的anchor
                 
        extractor, classifier = decom_vgg16() #conv5_3及之前的部分，分类器

        rpn = RegionProposalNetwork(
            512, 512,
            ratios=ratios,
            anchor_scales=anchor_scales,
            feat_stride=self.feat_stride,
        ) #返回rpn_locs, rpn_scores, rois, roi_indices, anchor

        head = VGG16RoIHead(
            n_class=n_fg_class + 1,
            roi_size=7,
            spatial_scale=(1. / self.feat_stride),
            classifier=classifier
        ) #下面会分析VGG16RoIHead（），n_class = 21（加上背景）

        super(FasterRCNNVGG16, self).__init__(
            extractor,
            rpn,
            head,
        ) #相当于给faster_rcnn传入参数extractor, rpn, head


class VGG16RoIHead(nn.Module):
    """Faster R-CNN Head for VGG-16 based implementation.
    This class is used as a head for Faster R-CNN.
    This outputs class-wise localizations and classification based on feature
    maps in the given RoIs.
    
    Args:
        n_class (int): The number of classes possibly including the background.
        roi_size (int): Height and width of the feature maps after RoI-pooling.
        spatial_scale (float): Scale of the roi is resized.
        classifier (nn.Module): Two layer Linear ported from vgg16
    """

    def __init__(self, n_class, roi_size, spatial_scale,
                 classifier):
        # n_class includes the background
        super(VGG16RoIHead, self).__init__()

        self.classifier = classifier #vgg16中的classifier
        self.cls_loc = nn.Linear(4096, n_class * 4)
        self.score = nn.Linear(4096, n_class)

        normal_init(self.cls_loc, 0, 0.001)
        normal_init(self.score, 0, 0.01) #全连接层权重初始化

        self.n_class = n_class #加上背景21类
        self.roi_size = roi_size #7
        self.spatial_scale = spatial_scale # 1/16
        #将大小不同的roi变成大小一致，得到pooling后的特征，大小为[300, 512, 7, 7]。.利用Cupy实现在线编译的
        #使用的是from torchvision.ops import RoIPool
        self.roi = RoIPool( (self.roi_size, self.roi_size),self.spatial_scale) 
    def forward(self, x, rois, roi_indices):
        """Forward the chain.
        We assume that there are :math:`N` batches.
        Args:
            x (Variable): 4D image variable.
            rois (Tensor): A bounding box array containing coordinates of
                proposal boxes.  This is a concatenation of bounding box
                arrays from multiple images in the batch.
                Its shape is :math:`(R', 4)`. Given :math:`R_i` proposed
                RoIs from the :math:`i` th image,
                :math:`R' = \\sum _{i=1} ^ N R_i`.
            roi_indices (Tensor): An array containing indices of images to
                which bounding boxes correspond to. Its shape is :math:`(R',)`.
        """
        # in case roi_indices is  ndarray
        roi_indices = at.totensor(roi_indices).float() #ndarray->tensor
        rois = at.totensor(rois).float()
        indices_and_rois = t.cat([roi_indices[:, None], rois], dim=1)
        # NOTE: important: yx->xy
        xy_indices_and_rois = indices_and_rois[:, [0, 2, 1, 4, 3]]
        indices_and_rois =  xy_indices_and_rois.contiguous() #把tensor变成在内存中连续分布的形式

        pool = self.roi(x, indices_and_rois) #ROIPOOLING
        pool = pool.view(pool.size(0), -1)
        fc7 = self.classifier(pool) #decom_vgg16（）得到的calssifier,得到4096
        roi_cls_locs = self.cls_loc(fc7) #（4096->84） 84=21*4
        roi_scores = self.score(fc7) #（4096->21）
        return roi_cls_locs, roi_scores


def normal_init(m, mean, stddev, truncated=False):
    """
    weight initalizer: truncated normal and random normal.
    """
    # x is a parameter
    if truncated:
        m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)  # not a perfect approximation
    else:
        m.weight.data.normal_(mean, stddev)
        m.bias.data.zero_()